# CIFAR-10 classifier from nnBuilder

This is a simple implementation of a CIFAR-10 classifier using the nnBuilder framework. Trying to reproduce the network of https://www.tensorflow.org/tutorials/deep_cnn/ (Work in progress)

In [1]:
import tensorflow as tf
import numpy as np
import sys
sys.path.append('../nnBuilder')
from nnLayer import *
from nnInput import *
from nnTrainer import *
from nnHandler import *
from _nnUtils import *

In [2]:
preprocessing=False #Slow, error rate not lowering?
data=Layer(x=None,type="CIFAR_10")
preprocess_layers=[]
preprocess_layers.append(dict(type="Batch_Random_Crop",shape=[24,24,3]))
preprocess_layers.append(dict(type="Batch_Random_Horizontal_Flip"))
preprocess_layers.append(dict(type="Batch_Random_Brightness"))
preprocess_layers.append(dict(type="Batch_Random_Contrast"))
preprocess_layers.append(dict(type="Batch_Whitening"))
preprocess_layers.append(dict(type="Pipeline",num_threads=8))
batch=Layer(x=data,type="Batch_Slice",batch=128,out_features=preprocessing and preprocess_layers or None)
layers=[]
layers.append(dict(type="Convolution",pad="SAME",window=5,stride=1,size=64,relu=True,out_features=[
              dict(type="Pool",pad="SAME",window=3,stride=2,pool_type="max"),
              dict(type="Local_Response_Norm")]))
layers.append(dict(type="Convolution",pad="SAME",window=5,stride=1,size=64,relu=True,out_features=[
              dict(type="Pool",pad="SAME",window=3,stride=2,pool_type="max"),
              dict(type="Local_Response_Norm")]))
layers.append(dict(type="Relu",size=384))
layers.append(dict(type="Relu",size=192))
layers.append(dict(type="Linear",size=10,in_features=["Dropout"]))
network_def=dict(type="Network",layers=layers)
network=Layer(x=batch,**network_def)
trainer=ClassifierTrainer(network=network,optimizer="adam",finish=True)
sess=SessManager(data,batch,network,trainer)#,input_network)
sess.start()

Starting new session


In [3]:
trainer.train(0,keep_rate=1.)
trainer.eval_error()
for i in range(50):
    trainer.train(100,keep_rate=1.)
    trainer.train(0,keep_rate=1.)
    trainer.eval_error() #Training error, reaches about 10% without preprocessing

Error rate (train): 0.901563
Error rate (train): 0.646094
Error rate (train): 0.520312
Error rate (train): 0.519531
Error rate (train): 0.467969
Error rate (train): 0.428906
Error rate (train): 0.403125
Error rate (train): 0.373437
Error rate (train): 0.386719
Error rate (train): 0.380469
Error rate (train): 0.357812
Error rate (train): 0.330469
Error rate (train): 0.35
Error rate (train): 0.339844
Error rate (train): 0.301562
Error rate (train): 0.317969
Error rate (train): 0.290625
Error rate (train): 0.285156
Error rate (train): 0.246094
Error rate (train): 0.275781
Error rate (train): 0.251563
Error rate (train): 0.278125
Error rate (train): 0.282031
Error rate (train): 0.220313
Error rate (train): 0.252344
Error rate (train): 0.228125
Error rate (train): 0.225781
Error rate (train): 0.222656
Error rate (train): 0.235938
Error rate (train): 0.2125
Error rate (train): 0.19375
Error rate (train): 0.220313
Error rate (train): 0.2
Error rate (train): 0.238281
Error rate (train): 0.1726

In [4]:
network.save() #The full network definition
save=network.save() #The full network definition
assert(Layer(x=batch,**save).save()==save) #Consistency
print(save==network_def) #Saving adds stuff, should still be equivalent
save

False


{'layers': [{'input_channels': None,
   'out_features': [{'pad': 'SAME',
     'pool_type': 'max',
     'stride': 2,
     'type': 'Pool',
     'window': 3},
    {'type': 'Local_Response_Norm'}],
   'pad': 'SAME',
   'rand_scale': 0.1,
   'relu': True,
   'size': 64,
   'stride': 1,
   'type': 'Convolution',
   'window': 5},
  {'input_channels': None,
   'out_features': [{'pad': 'SAME',
     'pool_type': 'max',
     'stride': 2,
     'type': 'Pool',
     'window': 3},
    {'type': 'Local_Response_Norm'}],
   'pad': 'SAME',
   'rand_scale': 0.1,
   'relu': True,
   'size': 64,
   'stride': 1,
   'type': 'Convolution',
   'window': 5},
  {'rand_scale': 0.1, 'size': 384, 'type': 'Relu'},
  {'rand_scale': 0.1, 'size': 192, 'type': 'Relu'},
  {'in_features': [{'type': 'Dropout'}],
   'rand_scale': 0.1,
   'size': 10,
   'type': 'Linear'}],
 'type': 'Network'}